In [1]:
import os
if 'Conformal-Sparsemax/notebooks' in os.getcwd():
    os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from conformal_sparsemax.classifier import CNN, CNN_CIFAR, get_data,train,evaluate

/home/joaocalem/.pyenv/versions/3.10.6/envs/stable-diffusion/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/joaocalem/.pyenv/versions/3.10.6/envs/stable-diffusion/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
import torch
from torch import nn
from sklearn.metrics import f1_score

In [4]:
from entmax.losses import SparsemaxLoss, Entmax15Loss

In [5]:
loss = 'sparsemax' #sparsemax or softmax
dataset = 'CIFAR100' #CIFAR100 or MNIST

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if loss == 'sparsemax':
    criterion = SparsemaxLoss()
elif loss == 'entmax15':
    criterion = Entmax15Loss()
elif loss == 'softmax':
    criterion = torch.nn.NLLLoss()
train_dataloader, dev_dataloader, test_dataloader, _ = get_data(0.2,16,dataset = dataset)

if dataset == 'CIFAR100':
    model = CNN_CIFAR(loss).to(device)
elif dataset == 'MNIST':
    model = CNN(loss,n_classes=10,input_size=28,channels=1).to(device)
else:
    raise Exception('Wrong dataset name')
    
model, train_history, val_history, f1_history = train(model,
                                            train_dataloader,
                                            dev_dataloader,
                                            criterion,
                                            epochs=350,
                                            patience=3)

_, predicted_labels, true_labels, test_loss = evaluate(
                                                    model,
                                                    test_dataloader,
                                                    criterion)

f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f'Test loss: {test_loss:.3f}')
print(f'Test f1: {f1:.3f}')

Files already downloaded and verified
Files already downloaded and verified
-- Epoch 1 --


2450it [00:33, 74.10it/s]


train_loss: 0.636
val_loss: 0.578
val_f1: 0.002
-- Epoch 2 --


2450it [00:34, 71.92it/s]


train_loss: 0.558
val_loss: 0.552
val_f1: 0.003
-- Epoch 3 --


2450it [00:34, 70.99it/s]


train_loss: 0.537
val_loss: 0.509
val_f1: 0.005
-- Epoch 4 --


580it [00:08, 70.34it/s]


KeyboardInterrupt: 

In [ ]:
results = {
    'train_history':train_history,
    'val_history':val_history,
    'f1_history':f1_history,
}

In [ ]:
import json
with open(f'results/{dataset}_{loss}_results.json', 'w') as f:
    json.dump(results, f)

In [ ]:
torch.save(model.state_dict(), f'models/{dataset}_{loss}.pth')